In [11]:
import numpy as np

import os
import sys
sys.path.append("../")

from IoTPy.core.stream import StreamArray, run
from IoTPy.helper_functions.recent_values import recent_values
from IoTPy.agent_types.simple import f_window, join_synch
from IoTPy.agent_types.merge import zip_stream
from IoTPy.agent_types.sink import sink_window

num_out_sample = [0]
        
def f(window, num_out_sample, data_size, output_stream):
    if num_out_sample[0] > 0:
        # Already putting elements of a sample
        num_out_sample[0] = num_out_sample[0] + 1
        if num_out_sample[0] >= data_size:
            # Finished outputting a sample of size data_size
            # Get ready to output the next sample.
            num_out_sample[0] = 0    
    else:
        # Not currently outputting a sample
        if np.mean(window[-data_size: ])> np.mean(window):
            # Start output of next sample because mean of
            # sample exceeds mean of long-term window
            output_stream.append(window[-data_size:])
            num_out_sample[0] = 1  

def g(v, out_stream): out_stream.append(v)

    
data_size=2
s0 = StreamArray(name="accelerometer_0", dimension=3, dtype=float)
s1 = StreamArray(name="accelerometer_1", dimension=3, dtype=float)
accelerometers = [s0, s1]
num_accelerometers = len(accelerometers)
x = StreamArray(name='merged_accelerometer_streams', 
                dimension=((num_accelerometers,3)), dtype=float)
y = StreamArray(name='signal', dimension=((data_size, num_accelerometers, 3)),
                dtype=float)

z = StreamArray(name='z', dimension=((data_size, num_accelerometers, 3)),
                dtype=float)


#zip_stream(in_streams=accelerometers, out_stream=x)
#sink_window(func=f, in_stream=x, window_size=3, step_size=1, 
           #num_out_sample=num_out_sample, data_size=data_size, output_stream=y)
join_synch(func=g, in_streams=accelerometers, out_stream=x)
#f_window(func=f, in_stream=x, window_size=3, step_size=1, 
           #num_out_sample=num_out_sample, data_size=data_size, output_stream=y))


s0.append(np.array([0.0, 1.0, 2.0]))
s1.append(np.array([3.0, 4.0, 5.0]))
run()
print("x is ", recent_values(x))
print("y is ", recent_values(y))


s0.append(np.array([10.0, 11.0, 12.0]))
s1.append(np.array([13.0, 14.0, 15.0]))
run()
print("x is ", recent_values(x))
print("y is ", recent_values(y))


s0.append(np.array([20.0, 21.0, 22.0]))
s1.append(np.array([23.0, 24.0, 25.0]))
run()
print("x is ", recent_values(x))
print("y is ", recent_values(y))


s0.append(np.array([30.0, 31.0, 32.0]))
s1.append(np.array([33.0, 34.0, 35.0]))
run()
print("x is ", recent_values(x))
print("y is ", recent_values(y))

AttributeError: 'tuple' object has no attribute 'shape'

In [2]:
def g(window, state, output_stream):
    print ('state starting is ', state)
    output_stream.append(sum(window)+state)
    state = sum(window)+state
    return state

xx = StreamArray(name='xx', dtype=int)
yy = StreamArray(name ='yy', dtype=int)
sink_window(func=g, in_stream=xx, window_size=2, step_size=1, 
           state=0, output_stream=yy)


xx.append(np.array(1))
run()
print("xx is ", recent_values(xx))
print("yy is ", recent_values(yy))



xx.append(np.array(2))
run()
print("xx is ", recent_values(xx))
print("yy is ", recent_values(yy))


xx.append(np.array(3))
run()
print("xx is ", recent_values(xx))
print("yy is ", recent_values(yy))




xx is  [1]
yy is  []
state starting is  0
xx is  [1 2]
yy is  [3]
state starting is  3
xx is  [1 2 3]
yy is  [3 8]
